<a href="https://colab.research.google.com/github/lawun330/FCC-ML-Journey/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-06-08 13:37:02--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   102MB/s    in 0.2s    

2024-06-08 13:37:03 (102 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
user_rating_frequency = df_ratings.groupby(by='user')['rating'].count().reset_index().rename(columns={'rating':'user_rating_frequency'})
U = user_rating_frequency.to_numpy() # change it to numpy array
U = U[U[:,1].argsort()] # U[:,1].argsort() returns the indices in sorted order
user_rating_frequency = pd.DataFrame(U, columns=['user','user_rating_frequency']) # rebuild the dataframe object from numpy array
U = user_rating_frequency[user_rating_frequency['user_rating_frequency']>=200] # remove users who rate less than 200 times
original_idx = U['user'].values # get the users of the filtered data
df_URF = df_ratings[df_ratings['user'].isin(original_idx)] # build the sorted, filtered dataframe with isbn values
count = df_ratings['isbn'].value_counts()
original_indices = count[count>=100].index
df_UBTR = df_ratings[df_ratings['isbn'].isin(original_indices)]  # isbn values will be used as a joint to connect dataframes
df_processed_ratings = pd.merge(df_URF, df_UBTR) # new df_ratings
df = pd.merge(df_books, df_processed_ratings) # final df
df = df.drop_duplicates(['title','user']) # removing duplicates
piv=df.pivot(index='title', columns='user', values='rating').fillna(0)

book_titles = piv.index
matrix = piv.values

model = NearestNeighbors(n_neighbors=5, algorithm="brute", metric="cosine") # build the model
model.fit(matrix) # train the model

NearestNeighbors(algorithm='brute', metric='cosine')

In [5]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  if book!="":
        recommended_books = []
        similar_books = []
        recommended_books.append(book) # add the input book as 1st entry

        if book in book_titles:
            a_book_df = piv[book_titles == book] # get the input book dataframe
            rating_array = a_book_df.to_numpy().reshape(1, -1) # convert df to np # reshape the numpy array to 1 row with unknown columns
            distances, indices  = model.kneighbors(rating_array, n_neighbors=6) # get (the book itself + 5 closest neighbors') distances and indices

            for index in indices[0]: # can also use indices.flatten()
                similar_book_name = piv.index[index]
                if similar_book_name == book: # skip the book itself
                    continue
                # np.where(np_array==value) returns the index(in array form) of that value in np_array # [0][0] for un-nesting # can also use len(np_array.flatten())
                similarity = distances[0][np.where(indices[0]==index)[0][0]] # distances[0][i] to get the distance
                similar_books.append([similar_book_name, similarity]) # book_name-distance pair is listed and add to the similar_books
            similar_books.reverse()

        recommended_books.append(similar_books) # add similar_books as 2nd entry
  return recommended_books

In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
